This notebook implements the AIR-to-CCS conversion algorithm, found in the canonical [CCS paper](https://eprint.iacr.org/2023/552), $\text{Lemma 3}$. We apply several modification to this lemma, since we have 
found *potential* inconsistencies in the paper, all described in [this hackmd document](https://hackmd.io/5f2W4cGsQYy3W_rqVek3yA?view).

How to enforce boundary constraints is not specified in the CCS paper. We apply here the algorithm described in the above cited document.

We will apply the air-to-ccs transformation to a simple fibonacci air circuit. The trace table for our fibonacci circuit is:

| `c_0` | `c_1` |
|---|---|
| 1  | 1 |
| 2 | 3 |
| 5 | 8 |
| 13 | 21 |

This corresponds to outputting the 7th and 8th fibonacci numbers. 

**Transition constraints**


$$ g_0(x_0, x_1, x_2, x_3) = x_2 - x_1 - x_0 $$
$$ g_1(x_0, x_1, x_2, x_3) = x_3 - x_2 - x_1 $$

**Boundary constraints**


$$ T[0][0] - 1 = 0 $$
$$ T[0][1] - 1 = 0 $$
$$ T[3][0] - 13 = 0 $$
$$ T[3][1] - 21 = 0 $$

where $T$ is the trace table.

In [81]:
p = 21888242871839275222246405745257275088696311157297823662689037894645226208583
F = GF(p)
P.<x0, x1, x2, x3> = F[]
g0 = x2 - x1 - x0
g1 = x3 - x2 - x1

# get random element and compute single g polynomial
r = F.random_element(-10, 10)
g = r^0 * g0 + r * g1


trace = [[1, 1], [2, 3], [5, 8], [13, 21]]

m, t, d, q = len(trace) - 1, len(trace[0]) * 2, g.degree(), len(g.monomials())
n = (m+1) * t/2 + 1

len_w_air = n - t - 1


In [93]:
def ccs_is_satisfied(F, z, matrices, multisets, constants):
    satisfied_instance_witness = vector(F, [0 for i in range(matrices[0].dimensions()[0])])
    z_final = vector(F, [0 for i in range(matrices[0].dimensions()[0])])
    for i, c in enumerate(constants):
        multiset = multisets[i]
        z_i = vector(F, [1 for i in range(matrices[0].dimensions()[0])])
        for j in multiset:
            z_i = z_i.pairwise_product(matrices[j] * z)
        z_final += c * z_i
    return z_final == satisfied_instance_witness
    

In [82]:
# deriving M_0, ..., M_{t-1} and N
N = 0
matrices = [matrix(F, m, n) for i in range(t)]
for i in range(m):
    for j in range(t):
        matrix_to_index = matrices[j]
        k_j = (t*(i-1)/2) + j 
        if i == 0 and j < t/2:
            matrix_to_index[i, j + len_w_air] = 1
            continue
        elif i == m - 1 and j >= t/2:
            matrix_to_index[i, j + len_w_air] = 1
        else:
            matrix_to_index[i, k_j] = 1
        N += 1

In [84]:
# Deriving S_0, ..., S_{q-1} and c_0, ..., c_{q-1}
multisets = [[] for i in range(q)]
c_coefficients = g.coefficients() # returns in the order of the monomials
g_monomials = g.monomials()

for i, monomial in enumerate(g.monomials()):
    # we extracted the coefficient above
    monomial_variables_degrees, monomial_coeff = list(monomial.dict().items())[0] # [(deg x0, deg x1, ...), coeff]
    for j, degree in enumerate(monomial_variables_degrees):
        if degree != 0:
            for _ in range(degree):
                # append j with multiplicity equal to the degree of the variable
                multisets[i].append(j)

In [96]:
z = vector(F, trace[1] + trace[2] + trace[0] + trace[3] + [1])
assert ccs_is_satisfied(F, z, matrices, multisets, c_coefficients)
z[0] = 1
assert ccs_is_satisfied(F, z, matrices, multisets, c_coefficients) == False